# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
from tabulate import tabulate
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)    
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

267


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin Apache Cassandra code.

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS eventdata 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('eventdata')
except Exception as e:
    print(e)

## Sample Queries and Create, Insert, Retrieve Operation

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 172, and itemInSession  = 4
`SELECT artist, title, song FROM session_event WHERE sessionId=172 and itemInSession=4`

Since we use sessionId as the main identifier alongside itemInSession as the composite, <br>
Partition Key should be the sessionId follows by itemInSession as the Clustering Columns.

In [8]:
query = "CREATE TABLE IF NOT EXISTS song_by_session_item "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        #print(line)
        query = "INSERT INTO song_by_session_item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted

In [10]:
query = "select artist, song, length from song_by_session_item WHERE sessionId=172 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = []
for row in rows:
    df.append([row.artist, row.song, row.length])

column_labels = ['artist', 'song', 'length']
result = pd.DataFrame(df, columns=column_labels)
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+--------------+---------------------+----------+
|    | artist       | song                |   length |
|----+--------------+---------------------+----------|
|  0 | Brett Dennen | World Keeps Turning |   179.67 |
+----+--------------+---------------------+----------+


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
`SELECT artist, song, firstName, lastName FROM song_by_user_session WHERE userId=10 AND sessionId=182 SORT BY itemInSession`

userId is likely the main identifier as the Partition Key due to many duplicate sessionId. <br>
This way, we use sessionId as the Cluster Columns with itemInSession to sort naturally.

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_by_user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_by_user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted

In [13]:
query = "select artist, song, firstName, lastName from song_by_user_session WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = []
for row in rows:
    df.append([row.artist, row.song, row.firstname, row.lastname])

column_labels = ['artist', 'song', 'firstname', 'lastname']
result = pd.DataFrame(df, columns=column_labels)
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+-------------------+------------------------------------------------------+-------------+------------+
|    | artist            | song                                                 | firstname   | lastname   |
|----+-------------------+------------------------------------------------------+-------------+------------|
|  0 | Down To The Bone  | Keep On Keepin' On                                   | Sylvie      | Cruz       |
|  1 | Three Drives      | Greece 2000                                          | Sylvie      | Cruz       |
|  2 | Sebastien Tellier | Kilometer                                            | Sylvie      | Cruz       |
|  3 | Lonnie Gordon     | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie      | Cruz       |
+----+-------------------+------------------------------------------------------+-------------+------------+


### 3. Give me every user name (first and last) in my music app history who listened to the song 'Come As You Are'
`SELECT firstName, lastName FROM user_by_song WHERE song='Come As You Are'`

We only use song as the Partition Key since we only need to filter the song. <br>
userId is set as the Cluster Columns for identifying the name efficiently.

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_by_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted

In [16]:
query = "select firstName, lastName from user_by_song WHERE song='Come As You Are'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = []
for row in rows:
    df.append([row.firstname, row.lastname])

column_labels = ['firstname', 'lastname']
result = pd.DataFrame(df, columns=column_labels)
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+-------------+------------+
|    | firstname   | lastname   |
|----+-------------+------------|
|  0 | Lily        | Koch       |
+----+-------------+------------+


### Drop the tables before closing out the sessions

In [17]:
query = "drop table song_by_session_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "drop table song_by_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "drop table user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()